## Load data

In [1]:
file_test = 'C:\Data\Miroculus\A3_1_2016_text_inputs\A3_1_2016_binary_data_entities_normalize_rmstopwords_stem_test.tsv'
file_train = 'C:\Data\Miroculus\A3_1_2016_text_inputs\A3_1_2016_binary_data_entities_normalize_rmstopwords_stem_train.tsv'

In [2]:
import pandas as pd
test = pd.read_csv(file_test, quotechar='"', delimiter='\t', header=None, names=['Label', 'Text'])
train = pd.read_csv(file_train, quotechar='"', delimiter='\t', header=None, names=['Label', 'Text'])
test.head; train.head

<bound method NDFrame.head of              Label                                               Text
0      NO_RELATION  result identifi previous report breast cancer-...
1      NO_RELATION  conclus taken togeth mmvarentty1mm/ggvarentty1...
2         RELATION  revers correl mmvarentty1mm ggvarentty1gg also...
3         RELATION  result suggest recognit site mmvarentty1mm mmv...
4         RELATION  issu jci ghosh colleagu identifi uniqu microrn...
5      NO_RELATION  conclus mmvarentty1mm may promot cell ggvarent...
6      NO_RELATION        howev role mmvarentty1mm npc remain unknown
7         RELATION  targetscan predict ggvarentty1gg ggvarentty2gg...
8      NO_RELATION  increas mmvarentty1mm depend upon ggvarentty1g...
9      NO_RELATION  revers correl mmvarentty1mm ggvarentty1gg expr...
10        RELATION  studi identifi mmvarentty1mm mmvarentty2mm mic...
11        RELATION  show mmvarentty1mm oncosuppress microrna lost ...
12     NO_RELATION  newli identifi mmvarentty1mm/ggvarentty1

## Vectorise text data into numerical vector

### Using `CountVectorizer`

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
tf_vectorizer = CountVectorizer(analyzer='word', stop_words='english', ngram_range=(1,3), max_df=0.9, min_df=0.01)
train_tf_vectors = tf_vectorizer.fit_transform(train.Text)

** Note: ** Numbers not removed. Consider removing numbers.

`tf_vectorizer.vocabulary_` is a `dict`

In [4]:
tf_vectorizer.vocabulary_

{'001': 0,
 '05': 1,
 '16': 2,
 '3p': 3,
 '5p': 4,
 'act': 5,
 'activ': 6,
 'addit': 7,
 'affect': 8,
 'aim': 9,
 'alter': 10,
 'analys': 11,
 'analysi': 12,
 'analyz': 13,
 'apoptosi': 14,
 'arrest': 15,
 'assay': 16,
 'associ': 17,
 'bind': 18,
 'bioinformat': 19,
 'biomark': 20,
 'blot': 21,
 'breast': 22,
 'breast cancer': 23,
 'cancer': 24,
 'cancer cell': 25,
 'cancer tissu': 26,
 'carcinoma': 27,
 'caus': 28,
 'cell': 29,
 'cell cycl': 30,
 'cell growth': 31,
 'cell line': 32,
 'cell migrat': 33,
 'cell prolifer': 34,
 'chain': 35,
 'chain reaction': 36,
 'chang': 37,
 'clinic': 38,
 'cluster': 39,
 'colon': 40,
 'combin': 41,
 'compar': 42,
 'conclus': 43,
 'conclus mmvarentty1mm': 44,
 'confirm': 45,
 'contribut': 46,
 'control': 47,
 'correl': 48,
 'correl ggvarentty1gg': 49,
 'correl mmvarentty1mm': 50,
 'crc': 51,
 'cycl': 52,
 'data': 53,
 'decreas': 54,
 'decreas express': 55,
 'demonstr': 56,
 'demonstr mmvarentty1mm': 57,
 'depend': 58,
 'detect': 59,
 'determin': 60,
 

In [5]:
train_tf_vectors.shape

(10783, 358)

In [6]:
train_tf_vectors.data

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [7]:
print(train_tf_vectors.nnz) # number of non-zero elements
print(train_tf_vectors.nnz / float(train_tf_vectors.shape[0])) # non-zero per sample
print(train_tf_vectors.nnz / float(train_tf_vectors.shape[0] * train_tf_vectors.shape[1]) * 100) # non-zero per feature space in %

134461
12.469720856904386
3.4831622505319517


## Data Preprocessing: 
### Attributes and Target

### Convert categorical labels into numerical labels

In [8]:
from sklearn import preprocessing

In [9]:
le = preprocessing.LabelEncoder()
le.fit(train.Label)
list(le.classes_)

['NO_RELATION', 'RELATION']

In [10]:
y_train = le.transform(train.Label);
y_test = le.transform(test.Label);

print(y_train[0:5])
print(le.inverse_transform(y_train[0:5]))

[0 0 1 1 1]
['NO_RELATION' 'NO_RELATION' 'RELATION' 'RELATION' 'RELATION']


`X` is data, `y` is target. `X` has the size of `n_samples` x `n_features`, `y` has `n_samples` x 1 labels

In [11]:
X_train = train_tf_vectors

In [12]:
X_test = tf_vectorizer.transform(test.Text)

In [13]:
X_test.shape

(3595, 358)

In [14]:
X_test.data

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

## Log Res: 

In [15]:
from sklearn import linear_model
clf = linear_model.LogisticRegression(C=1e5, class_weight='balanced')
clf.fit(X_train, y_train)

LogisticRegression(C=100000.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [16]:
y_pred = clf.predict(X_test)

In [17]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.72      0.73      0.73      1804
          1       0.72      0.72      0.72      1791

avg / total       0.72      0.72      0.72      3595



In [18]:
print(metrics.confusion_matrix(y_test, y_pred))

[[1315  489]
 [ 503 1288]]


### Coeficients of the decision function

In [19]:
clf.coef_

array([[  7.09684983e-02,  -2.23268453e-01,   9.36630528e-01,
         -1.08692925e-02,  -3.01619299e-01,   5.71774388e-02,
          1.73925777e-01,   1.40609072e-01,  -1.25286873e-02,
         -4.19645500e-01,   2.41183276e-01,  -4.22297254e-02,
         -1.22458748e-01,  -8.47247067e-02,  -2.76198258e-02,
         -2.40918715e-01,  -8.22299227e-02,  -2.34019431e-01,
          1.34192290e-01,   4.53468907e-01,  -5.19034884e-01,
          1.39043842e+00,  -1.67989528e-01,   1.67983944e-01,
         -3.89759623e-01,  -4.73098220e-02,   4.02777762e-01,
         -5.48829229e-01,   3.52264404e-01,  -4.40250675e-02,
          6.13257539e-01,  -9.61261102e-02,   7.53633532e-01,
          7.32300075e-02,  -6.01412379e-02,   6.42242153e-01,
         -1.26948904e+00,  -1.72678240e-01,  -2.12807946e-01,
          2.65698220e-01,   2.29032332e-01,   1.18880983e-01,
         -2.26347803e-01,   7.31082626e-02,  -7.83880775e-04,
          1.57030118e-01,   7.02103736e-02,   5.76367747e-02,
        

In [20]:
len(clf.coef_[0,])

358

Get the index of coef with the maximum value.

In [21]:
clf.coef_.argmax()

355

In [22]:
m = clf.coef_.argmax()
print(list(tf_vectorizer.vocabulary_.keys())[list(tf_vectorizer.vocabulary_.values()).index(m)])

western


Get the index of the highest 10 coef value.

**Note** `clf.coef_` is ndarray, see below:

In [23]:
clf.coef_[0]

array([  7.09684983e-02,  -2.23268453e-01,   9.36630528e-01,
        -1.08692925e-02,  -3.01619299e-01,   5.71774388e-02,
         1.73925777e-01,   1.40609072e-01,  -1.25286873e-02,
        -4.19645500e-01,   2.41183276e-01,  -4.22297254e-02,
        -1.22458748e-01,  -8.47247067e-02,  -2.76198258e-02,
        -2.40918715e-01,  -8.22299227e-02,  -2.34019431e-01,
         1.34192290e-01,   4.53468907e-01,  -5.19034884e-01,
         1.39043842e+00,  -1.67989528e-01,   1.67983944e-01,
        -3.89759623e-01,  -4.73098220e-02,   4.02777762e-01,
        -5.48829229e-01,   3.52264404e-01,  -4.40250675e-02,
         6.13257539e-01,  -9.61261102e-02,   7.53633532e-01,
         7.32300075e-02,  -6.01412379e-02,   6.42242153e-01,
        -1.26948904e+00,  -1.72678240e-01,  -2.12807946e-01,
         2.65698220e-01,   2.29032332e-01,   1.18880983e-01,
        -2.26347803e-01,   7.31082626e-02,  -7.83880775e-04,
         1.57030118e-01,   7.02103736e-02,   5.76367747e-02,
        -8.90550210e-02,

In [24]:
print(clf.coef_[0][0])
print(clf.coef_[0][1])

0.0709684983476
-0.22326845257


In [25]:
import numpy as np
topN = np.argsort(clf.coef_[0])[::-1][:20]
print(topN)

[355  21 162 323 264 103 328   2  49 288 331  67 245  32 114 330 287 160
  35 221]


In [26]:
for v in topN:
    print(list(tf_vectorizer.vocabulary_.keys())[list(tf_vectorizer.vocabulary_.values()).index(v)])


western
blot
invers
taken
prostat
gene ggvarentty1gg
target ggvarentty1gg
16
correl ggvarentty1gg
repress
target mmvarentty1mm
directli
partial
cell line
ggvarentty1gg protein
target ggvarentty2gg
report assay
interact
chain
mmvarentty3mm mmvarentty4mm mmvarentty5mm
